   <center><h1>This is a dev and test environment for EMD1.0</h1></center>

In [57]:
!pip install py4j
import py4j.java_gateway as jg
from py4j.java_gateway import JavaGateway
import subprocess
import threading
import os
import uuid
##############################################################################
'''Responsible for starting and stopping the NetLogo Controller Server'''
class NetLogo_Controller_Server_Starter:
    
    __gw = JavaGateway()# New gateway connection 
    ##server is in netlogo app folder
    __server_name = "C:/Program Files/NetLogo 6.0.2/app/NetLogoControllerServer.jar"
    
    '''Internal method to start JavaGateway server. Will be called by starServer on seperate thread'''
    def __runServer(self): 
        env = dict(os.environ)
        env['JAVA_OPTS'] = ''
        print(subprocess.call(['java', '-jar', self.__server_name]))
    
    '''Starts JavaGateway server'''
    def startServer(self):
        #Fire up the NetLogo Controller server through python
        thread = threading.Thread(target=self.__runServer, args=())
        thread.start()
        
    '''Send shutdown signal to the JavaGateway server. No further communication is possible unless server is restarted'''
    def shutdownServer(self, gw):
        self.__gw.shutdown(True)
##############################################################################

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [74]:
'''Responsible for communicating with the NetLogo controller, a Java executable'''
class NetLogo_Controller_Client:
    __bridge = None
    __gateway = JavaGateway()# New gateway connection 
    __session = None
    def __init__(self):
        self.__bridge = self.__gateway.entry_point
    
    '''Start up the model and run. Returns a single metric'''
    def run_model(self,model_string, steps, metric):
        
        
        session = self.__bridge.openModel(model_string)
        self.setParamsRandom(session, model_string)
        self.__bridge.command(session, "setup")
        self.__bridge.command(session, "repeat " + str(steps) + " [go]")
        result = self.__bridge.report(session, str(metric))
        self.__bridge.closeModel(session)#self.__bridge.refresh()
        return result
    
    
    
    '''Sets the parameters randomly through the JavaGateway'''
    def setParamsRandom(self, session, model_string):
        paramSpecs = self.__bridge.getParamList(session, model_string).getParamSpecs()

        for paramSpec in paramSpecs:
            if jg.is_instance_of(self.__gateway,paramSpec,"bsearch.space.DoubleDiscreteSpec"):
                paramValue = paramSpec.generateRandomValue(self.__gateway.jvm.org.nlogo.api.MersenneTwisterFast())
            if jg.is_instance_of(self.__gateway,paramSpec,"bsearch.space.DoubleContinuousSpec"):
                paramValue = paramSpec.generateRandomValue(self.__gateway.jvm.org.nlogo.api.MersenneTwisterFast())
            if jg.is_instance_of(self.__gateway,paramSpec,"bsearch.space.CategoricalSpec"):
                paramValue = paramSpec.generateRandomValue(self.__gateway.jvm.org.nlogo.api.MersenneTwisterFast())
                if type(paramValue) != bool:#isinstance(data[i][k], bool)
                    paramValue = '"{}"'.format(paramSpec.generateRandomValue(self.__gateway.jvm.org.nlogo.api.MersenneTwisterFast()))
            if jg.is_instance_of(self.__gateway,paramSpec,"bsearch.space.ConstantSpec"):
                paramValue = paramSpec.generateRandomValue(self.__gateway.jvm.org.nlogo.api.MersenneTwisterFast())
            print("set " + str(paramSpec.getParameterName()) + " " + str(paramValue))
            self.__bridge.command(session, "set " + str(paramSpec.getParameterName()) + " " + str(paramValue))
        self.__bridge.command(session, "set " + str(paramSpec.getParameterName()) + " " + str(paramValue))
        self.__bridge.command(session, "set historic-view? true" )
    
    '''Returns the Parameter Specification of the NetLogo Model'''
    def getParamSpecs(self):
        return self.__bridge.getParamList(session, model_string).getParamSpecs()
##############################################################################
'''Responsible for Writing Customized NetLogo models for EMD'''
class NetLogo_Writer:
    __original_model_path = ""
    __rule_injected_model_path = ""
    EMD_line = -1000
    
    '''locate the NetLogo file, read it in, and identify the line to be experimented with as EMD_line.'''
    def __init__(self, model_string):
        __original_model_path = model_string
        #find EMD entry point
        with  open(__original_model_path, 'r') as file_reader:#should probably catch an exception here

            for i, line in enumerate(file_reader):
                if ";;insert evolutionary code here" in line:
                    print(i)
                    print(line)
                    EMD_line = i + 1
                
    '''if EMD annotation exists, then inject the new rule string'''
    def injectNewRule (self, new_rule):
        # with is like your try .. finally block in this case
        with open(__original_model_path, 'r') as file:  #should probably catch an exception here
            # read a list of lines into data
            data = file.readlines()
            file.close()
        __rule_injected_model_path
        if EMD_line >= 0:
            print( "Your line: " + data[EMD_line])
            data[EMD_line] = new_rule
            print(data)
            # and write everything back
            with open(__rule_injected_model_path, 'w') as file:
                file.writelines( data )
                file.flush()
                file.close()

In [59]:
'''The ABM with all its flexibility as EMD sees it'''
'''EMD can use this class to control, set parameters, query results from'''
'''and most importantly, change rules within the NetLogo model specified'''
class ABM_NetLogo:
    path_to_model = ""
    model_as_text = ""
    parameters = []
    __nlogo_writer = None 
    def __init__(self, path_to_model):
        self.path_to_model = path_to_model
        __nlogo_writer = NetLogo_Writer(path_to_model)
        #Read in the model to memory (instead of repeated disk reads)
        print()
    def inject_new_rule(self, new_rule):
        #locate tag comment
        __nlogo_writer.injectNewRule(new_rule)
        #Replace line under tag with new_rule
        
        #Write model 
        
        #Flush, close
        print()
    def set_parameters(self, parameter_values):
        print()
    def get_parameter_names(self):
        #Return parameter names
        print()
    def run_abm(self):
        #Run the ABM
        print()


class EMD_Individual:
    def __init__(self):
        print()
    

In [45]:
#You can use this to start the server too... or just start it manually by running the controller jar and skip this
n = NetLogo_Controller_Server_Starter()
n.startServer()

In [53]:
modelpath = "./models/ArtificialAnasazi/Artificial Anasazi Ver 6.nlogo"
ABM_NetLogo(modelpath)

 259
    ;;insert evolutionary code here

()


<__main__.ABM_NetLogo instance at 0x0000000006226B08>

In [77]:
n = NetLogo_Controller_Client()
n.run_model("C:\Users\ch328575\Documents\Projects\EMD\EMD1.0\models\ArtificialAnasazi\Artificial Anasazi Ver 6.nlogo", 200, "count households")

set map-view "zones"
set harvest-variance 0.71
set death-age 40.0
set fertility-ends-age 30.0
set fertility 0.0103198368711
set historic-view? True
set harvest-adjustment 0.93
set base-nutrition-need 109.0
set min-death-age 44.0
set death-age-span 68.0
set min-fertility-ends-age 59.0
set fertility-ends-age-span 88.0
set min-fertility 21.9044408597
set fertility-span 0.9
set maize-gift-to-child 0.9
set water-source-distance 3.0


652.0